<a href="https://colab.research.google.com/github/natandovale/topic-in-big-data/blob/main/Suicidios_Sus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Baixando libs e dependências**


In [ ]:
!pip install cufflinks
!pip install plotly

In [2]:
import pandas as pd
import plotly
import plotly.offline as py
import plotly.graph_objs as go # criará de fato os gráficos
from plotly.offline import plot, iplot
import cufflinks as cf # para conectar o plotly ao pandas
cf.go_offline()
plotly.offline.init_notebook_mode(connected = True)

In [ ]:
import plotly.io as pio
pio.renderers

In [4]:
pio.renderers.default = 'colab'

In [7]:
arquivo = '/content/drive/MyDrive/Colab Notebooks/datasusSuicidios_2014_2018.xlsx'
df = pd.read_excel(arquivo, header=0)

In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Perguntas aos Dados



*   Quantas mortes houve no período de 2014 a 2018?
*   Quantos suicidios houve por estado?
*   Qual a taxa de suicidios por idade?
*   Qual estado civil das pessos que se suicidaram?
*   Qual a caracteristica fenotíca das pessoas que se suicidaram?
*   Numero de suicidio entre homens e mulhere?
*   Ocupação da pessoa que se suicidou?



**Renomeiando colunas**

In [8]:
df = df.rename(columns={'Unnamed: 1': 'ID_ESTADO', 'Unnamed: 2': 'ESTADO', 'Unnamed: 3': 'ANO', 'Unnamed: 4': 'CIRCOBITO', 
                        'Unnamed: 5': 'DTOBITO', 'Unnamed: 6': 'DTNASC', 'Unnamed: 7': 'SEXO', 'Unnamed: 8': 'RACA_COR',
                        'Unnamed: 9':'ESTCIV', 'Unnamed: 10': 'ESC', 'Unnamed: 11':'OCUP', 'Unnamed: 12':'CIDADE', 'Unnamed: 13':'LOCAL',
                        'Unnamed: 14':'ASSISTMED','Unnamed: 15':'IDADE'})

In [9]:
df = df.drop('Unnamed: 0', axis=1)

In [10]:
df = df.drop(0)

In [ ]:
df.columns

In [ ]:
df.info()

**Excluindo campos nulos**

In [ ]:
df.isnull().sum()

In [14]:
df.dropna(inplace = True)

In [15]:
df.DTOBITO = pd.to_datetime(df.DTOBITO)

**Criando coluna 'Data_Mes_Obto'**

In [ ]:
df.info()

In [24]:
df['Data_Mes_Obto'] = df.DTOBITO.dt.to_period('M').astype(str)

In [25]:
mortes_por_mes = df.groupby(by='Data_Mes_Obto').DTOBITO.count()

In [53]:
df.tail()

,ID_ESTADO,ESTADO,ANO,CIRCOBITO,DTOBITO,DTNASC,SEXO,RACA_COR,ESTCIV,ESC,OCUP,CIDADE,LOCAL,ASSISTMED,IDADE,Data_Mes_Obto
58622,138,TO,2018,Suicídio,2018-02-01,1979-05-27 00:00:00,Masculino,Preta,União consensual,8 a 11 anos,MOTORISTA DE CARRO DE PASSEIO,Lizarda,Via pública,Não,39,2018-02
58631,138,TO,2018,Suicídio,2018-04-16,1989-11-17 00:00:00,Masculino,Branca,Solteiro,8 a 11 anos,DESEMPREGADO CRONICO,Palmas,Domicílio,Não,29,2018-04
58633,138,TO,2018,Suicídio,2018-05-15,1955-08-08 00:00:00,Feminino,Parda,Viúvo,Nenhuma,DONA DE CASA,Palmas,Hospital,Sim,63,2018-05
58634,138,TO,2018,Suicídio,2018-07-24,2001-06-27 00:00:00,Masculino,Parda,Solteiro,8 a 11 anos,ESTUDANTE,Araguaína,Outros,Não,17,2018-07
58635,138,TO,2018,Suicídio,2018-12-28,1976-07-17 00:00:00,Masculino,Preta,União consensual,Nenhuma,0,Natividade,Domicílio,Não,42,2018-12


In [ ]:
type(mortes_por_mes)

In [27]:
# Criando os dados de dispersão
data = [go.Scatter(
    x=mortes_por_mes.index,
    y=mortes_por_mes.values,
    marker={'color': 'black'}
)]

configuracoes_layout = go.Layout(
    yaxis={'title': 'Número de Mortes', 'automargin': True, 'tickfont': {'size': 22}},  # Aumentando o tamanho da fonte dos rótulos do eixo y
    xaxis={'title': 'Período', 'automargin': True, 'tickfont': {'size': 22}}  # Aumentando o tamanho da fonte dos rótulos do eixo x
)

# Criando a figura do gráfico
fig = go.Figure(data=data, layout=configuracoes_layout)

# Plotando o gráfico
py.iplot(fig)

**Graficos mortes por estados**

In [28]:
suicidios_por_estados = df.groupby(by='ESTADO').ESTADO.count()

In [29]:
suicidios_por_estados = suicidios_por_estados.sort_values()

In [30]:
suicidios_ultimos_estados = suicidios_por_estados.tail(13)

In [31]:
cores = ['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#FF00FF', '#00FFFF', '#800000', '#008000', '#000080',
         '#808000', '#FFA500', '#808080', '#800080']


In [32]:
type(suicidios_por_estados)

pandas.core.series.Series

In [33]:
# Criando os dados de barras com as cores definidas e ordem crescente
data = [go.Bar(
    x=suicidios_ultimos_estados.index,
    y=suicidios_ultimos_estados.values,
    marker=dict(color=cores)
)]

# Configurando o layout do gráfico
configuracoes_layout = go.Layout(
    yaxis={'title': 'Número de Mortes','automargin': True, 'tickfont': {'size': 22}},
    xaxis={'title': 'Estado','automargin': True, 'tickfont': {'size': 22}},
    bargap=0.2  # Espaçamento entre as barras
)


# Criando a figura do gráfico
fig = go.Figure(data=data, layout=configuracoes_layout)

# Plotando o gráfico
py.iplot(fig)

In [56]:
suicidios_por_estados_todos = df.groupby(by='ESTADO').ESTADO.count()
suicidios_por_estados_todos = suicidios_por_estados.sort_values()
# Criando os dados de barras com as cores definidas e ordem crescente
data = [go.Bar(
    x=suicidios_por_estados_todos.index,
    y=suicidios_por_estados_todos.values
)]

# Configurando o layout do gráfico
configuracoes_layout = go.Layout(
    yaxis={'title': 'Número de Mortes'},
    xaxis={'title': 'Estado'},
    bargap=0.2  # Espaçamento entre as barras
)

# Criando a figura do gráfico
fig = go.Figure(data=data, layout=configuracoes_layout)

# Plotando o gráfico
py.iplot(fig)

**Graficos mortes por idades**

In [34]:
suicidios_por_idade = df.groupby(by='IDADE').IDADE.count()

In [35]:
data = [go.Bar(x=suicidios_por_idade.index,
                   y=suicidios_por_idade.values)]
 
configuracoes_layout = go.Layout(
                                 yaxis={'title':'Numero de Mortes'},
                                 xaxis={'title': 'Periodo'})

fig = go.Figure(data=data, layout=configuracoes_layout)

# plotando o grafico
py.iplot(fig)

In [52]:
# Encontrando as três maiores barras
maiores_barras = suicidios_por_idade.nlargest(3)

#Configurando o layout do gráfico
configuracoes_layout = go.Layout(
    yaxis={'title': 'Número de Mortes','automargin': True, 'tickfont': {'size': 30}},
    xaxis={'title': 'Idade','automargin': True, 'tickfont': {'size': 30}},
    barmode='group'  
)

#Criando a figura do gráfico
fig = go.Figure(data=data, layout=configuracoes_layout)

#Plotando o gráfico
py.iplot(fig)

In [ ]:
# Criando os dados de dispersão
data = [go.Scatter(
    x=mortes_por_mes.index,
    y=mortes_por_mes.values,
    marker={'color': 'black'}
)]

# Configurando o layout do gráfico
configuracoes_layout = go.Layout(
    title='Número de Mortes por Mês',
    yaxis={'title': 'Número de Mortes', 'automargin': True, 'tickfont': {'size': 14}},  
    xaxis={'title': 'Período', 'automargin': True, 'tickfont': {'size': 14}} 
)

# Criando a figura do gráfico
fig = go.Figure(data=data, layout=configuracoes_layout)

# Encontrando os três picos com o maior número de mortes
picos = mortes_por_mes.nlargest(3)

# Adicionando anotações aos três picos
annotations = []
for pico in picos.index:
    annotations.append(
        dict(
            x=pico,
            y=mortes_por_mes.loc[pico],
            text=str(mortes_por_mes.loc[pico]),
            showarrow=True,
            arrowhead=1,
            ax=0,
            ay=-30
        )
    )

fig.update_layout(annotations=annotations)

# Plotando o gráfico
py.iplot(fig)

In [ ]:
data = [go.Histogram(x=df['ESTCIV'])]
layout = go.Layout(title='Estado Civil', xaxis=dict(title='Estado Civil'), yaxis=dict(title='Número de Pessoas'))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [ ]:
#grafico3 = df['RACA_COR'].iplot(kind='hist', yTitle='Numero De Pessoas', xTitle='Cor')
data = [go.Histogram(x=df['RACA_COR'])]
layout = go.Layout(title='Características Fenotípicas', xaxis=dict(title='Cor'), yaxis=dict(title='Número de Pessoas'))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [ ]:
#df['SEXO'].iplot(kind='hist', yTitle='Numero De Pessoas', xTitle='SEXO')
data = [go.Histogram(x=df['SEXO'])]
layout = go.Layout(title='Suicidio entre homens e mulheres', xaxis=dict(title='Sexo'), yaxis=dict(title='Número de Pessoas'))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [ ]:
#df['ESTADO'].iplot(kind='hist', yTitle='Numero De Pessoas', xTitle='Estados')
data = [go.Histogram(x=df['ESTADO'])]
layout = go.Layout(title='Suicidios por Estados', xaxis=dict(title='Estados'), yaxis=dict(title='Número de Pessoas'))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [63]:
ocupacao_count = df['OCUP'].value_counts().head(10)

In [65]:
data = [go.Pie(
    labels=ocupacao_count.index,
    values=ocupacao_count.values
)]

# Configurando o layout do gráfico
layout = go.Layout(
    title='Ocupação'
)

# Criando a figura do gráfico
fig = go.Figure(data=data, layout=layout)

# Plotando o gráfico
py.iplot(fig)